In [3]:
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold # import KFold
from pandas.plotting import scatter_matrix
from sklearn.model_selection import cross_val_score
import os
from sklearn.model_selection import GridSearchCV   #Perforing grid search
from sklearn import metrics
import jupyter_slack
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


%env SLACK_ID=dhruvsand


os.environ['KMP_DUPLICATE_LIB_OK']='True'
%matplotlib inline  
random.seed(1998)

env: SLACK_ID=dhruvsand


In [4]:
train = pd.read_csv("./train_final.csv")
testorig = pd.read_csv("./test_final.csv")
test = testorig.loc[:,testorig.columns!='Id']
sample = pd.read_csv("./sample-submission.csv")


FileNotFoundError: [Errno 2] File b'./train_final.csv' does not exist: b'./train_final.csv'

In [4]:
Y= train.loc[:,train.columns=='Y']
X= train.loc[:,train.columns !='Y']
X= X.loc[:,X.columns !='Id']

X.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24
0,25884,1,33.63,118596,1,0,118595,125738,1,3,...,1945,118450,119184,1,121372,1,1,1,2,1
1,34346,1,10.62,118041,1,0,117902,130913,1,1,...,15385,117945,292795,1,259173,1,1,1,1,1
2,34923,1,1.77,118327,1,0,117961,124402,1,2,...,7547,118933,290919,1,118784,1,1,1,1,1
3,80926,1,30.09,118300,1,0,117961,301218,1,0,...,4933,118458,118331,1,307024,1,1,1,2,1
4,4674,1,1.77,119921,1,0,119920,302830,1,0,...,13836,142145,4673,1,128230,1,1,1,620,1


In [35]:
%%notify "KFold Done"

model = xgboost.XGBClassifier(learning_rate=0.001,n_estimators=30000, max_depth=25,min_child_weight=0,colsample_bytree=0.30,
                              subsample=0.85,colsample_bylevel=1,base_score=0.5,scale_pos_weight=1
                             ,max_delta_step=0,reg_alpha=0.15,reg_lambda=0,gamma=1e-5)

if False:
    kfold = KFold(n_splits=15)
    results = cross_val_score(model, X, Y.values.ravel(), cv=kfold,scoring='roc_auc',verbose=50,n_jobs=-1)
    print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [36]:
%%notify "N estimators and fit done"

if False:
    xgb_param = model.get_xgb_params()
    xgtrain = xgboost.DMatrix(X, label=Y.values.ravel())
    cvresult = xgboost.cv(xgb_param, xgtrain, num_boost_round=model.get_params()['n_estimators'], nfold=10,
                metrics='auc', early_stopping_rounds=300,verbose_eval=True,)
    model.set_params(n_estimators=cvresult.shape[0])
    print("N estimators" , cvresult.shape[0])
#Fit the algorithm on the data
model.fit(X, Y.values.ravel(),eval_metric='auc')



In [37]:
# model.fit(X,Y,eval_metric='auc')

submit = model.predict_proba(test)[:,1]
submit

array([0.9941282 , 0.96860886, 0.9999862 , ..., 0.9999832 , 0.99915934,
       0.99918014], dtype=float32)

In [16]:
%%notify "GridCV Done"

param_test6 = {
#  'max_delta_step':[i for i in range(0,10,1)],
#     'reg_lambda':[0,1e-3,0.1],
#     'reg_alpha':[0,1e-3,0.1],
    'gamma':[0.00001,0.00002,0.00003,0.00004,0.00005,0.00006,0.00007,0.00008,0.00009,0.0001]
#     'max_depth' :[8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25],
#     'min_child_weight':[0,1,2] 
#     'colsample_bytree':[0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8],
#     'subsample':[0.6,0.63,0.66,0.69,0.71,0.74,0.77,0.8]
#  'subsample':[0.84,0.85,0.86,0.87],
#  'colsample_bytree':[0.28,0.29,0.30,0.31,0.32]

    
}
gsearch6 = GridSearchCV(estimator = xgboost.XGBClassifier(eval_metric='auc',learning_rate=0.01,n_estimators=500,
                                  colsample_bytree=0.30,subsample=0.85,colsample_bylevel=1,reg_alpha=0.1
                                  ,base_score=0.5,scale_pos_weight=1,max_delta_step=0,reg_lambda=1e-7
                                  ,max_depth=25, min_child_weight=1,gamma=0.0002), 
 param_grid = param_test6, scoring='roc_auc',iid=False, cv=7, verbose=50,n_jobs=-1)
gsearch6.fit(X,Y.values.ravel())
gsearch6.cv_results_, gsearch6.best_params_, gsearch6.best_score_


# model2 = xgboost.XGBClassifier()
# kfold = KFold(n_splits=10, random_state=7)
# results = cross_val_score(model, X, Y, cv=kfold)
# print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))




Fitting 7 folds for each of 10 candidates, totalling 70 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   43.6s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   44.3s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   44.7s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   45.1s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  2.

In [17]:
gsearch6.cv_results_, gsearch6.best_params_, gsearch6.best_score_

({'mean_fit_time': array([36.30754372, 36.29152315, 36.56939115, 36.31759412, 36.57044196,
         36.86565716, 36.72442988, 36.51584693, 36.66590827, 33.74131894]),
  'std_fit_time': array([0.27718182, 0.3818532 , 0.56806264, 0.52729312, 0.34253358,
         0.34361033, 0.42542927, 0.51621328, 0.25571493, 5.2355605 ]),
  'mean_score_time': array([0.68579865, 0.77514352, 0.75102469, 0.68375724, 0.59719726,
         0.60798584, 0.65635582, 0.60813195, 0.56107695, 0.50912033]),
  'std_score_time': array([0.1471778 , 0.18544297, 0.22834281, 0.09656934, 0.11820723,
         0.13697974, 0.13376887, 0.09877369, 0.08514195, 0.09220018]),
  'param_gamma': masked_array(data=[1e-05, 2e-05, 3e-05, 4e-05, 5e-05, 6e-05, 7e-05, 8e-05,
                     9e-05, 0.0001],
               mask=[False, False, False, False, False, False, False, False,
                     False, False],
         fill_value='?',
              dtype=object),
  'params': [{'gamma': 1e-05},
   {'gamma': 2e-05},
   {'gamma':

In [38]:
submission = pd.DataFrame({'Id':testorig['Id'],'Y':submit})
submission.head()

,Id,Y
0,16384,0.994128
1,16385,0.968609
2,16386,0.999986
3,16387,0.999982
4,16388,0.999590


In [39]:
filename = 'submission1.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: submission1.csv
